In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import trange

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))
    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return x * (x > 0)

def relu_deriv(x):
    return x > 0

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [73]:
X, y = load_iris(return_X_y=True)

In [74]:
X = normalize(X)

In [75]:
y = to_one_hot(y)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [36]:
np.random.seed(42)
w0 = 2*np.random.random((4, 20)) - 1 # для входного слоя - 4 входа, 20 выхода
w1 = 2*np.random.random((20, 10)) - 1 # для внутреннего слоя - 20 входов, 10 выхода
w2 = 2*np.random.random((10, 3)) - 1 

In [77]:
# вариант с нормальным распределением: что-то похожее происходит в PyTorch в Linear
stdv = 1. / np.sqrt(4)
w0 = np.random.uniform(low=-stdv, high=stdv, size=(4, 20))
stdv = 1. / np.sqrt(20)
w1 = np.random.uniform(low=-stdv, high=stdv, size=(20, 10))
stdv = 1. / np.sqrt(10)
w2 = np.random.uniform(low=-stdv, high=stdv, size=(10, 3))

In [78]:
lr = 0.001

In [79]:
errors = []

for i in trange(900000):
    # прямое распространение(feed forward)
    layer0 = X_train
    layer1 = relu(np.dot(layer0, w0))
    layer2 = relu(np.dot(layer1, w1))
    layer3 = sigmoid(np.dot(layer2, w2))
    
    layer3_error = y_train - layer3
    layer3_delta = layer3_error * sigmoid_deriv(layer3)
    
    layer2_error = layer3_delta.dot(w2.T)
    layer2_delta = layer2_error * relu_deriv(layer2)
    
    layer1_error = layer2_delta.dot(w1.T)
    layer1_delta = layer1_error * relu_deriv(layer1)
    
    w2 += layer2.T.dot(layer3_delta) * lr
    w1 += layer1.T.dot(layer2_delta) * lr
    w0 += layer0.T.dot(layer1_delta) * lr
    
    error = np.mean(np.abs(layer3_error))
    errors.append(error)
    accuracy = (1 - error) * 100

100%|███████████████████████████████████████████████████████████████████████| 900000/900000 [00:49<00:00, 18300.54it/s]


In [80]:
trlayer0 = X_train
trlayer1 = relu(np.dot(trlayer0, w0))
trlayer2 = relu(np.dot(trlayer1, w1))
trlayer3 = sigmoid(np.dot(trlayer2, w2))
accuracy_score(y_train.argmax(axis=1), trlayer3.argmax(axis=1))

0.94

In [81]:
tlayer0 = X_test
tlayer1 = relu(np.dot(tlayer0, w0))
tlayer2 = relu(np.dot(tlayer1, w1))
tlayer3 = sigmoid(np.dot(tlayer2, w2))
accuracy_score(y_test.argmax(axis=1), tlayer3.argmax(axis=1))

1.0

### PyTorch implementation

In [83]:
train_x = torch.as_tensor(X_train, dtype=torch.float32)
test_x = torch.as_tensor(X_test, dtype=torch.float32)
train_y = torch.as_tensor(y_train.argmax(axis=1))
test_y = torch.as_tensor(y_test.argmax(axis=1))

train_ds = TensorDataset(train_x, train_y)
test_ds = TensorDataset(test_x, test_y)

batch_size = 8
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [84]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [85]:
model = nn.Sequential(
            nn.Linear(4, 20),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 3)
).to(device)

In [86]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # продвинутый SGD, динамически подстраивает Lr каждого параметра
epochs = 10000

In [87]:
train_acc, test_acc = 0, 0

for epoch in trange(epochs):
    model.train()
    running_train, running_test = 0, 0 
    for batch in train_dl:
        optimizer.zero_grad()
        x, y = batch
        x, y = x.to(device), y.to(device)
        out = model(x.to(device))
        loss = criterion(out, y.to(device))
        loss.backward()
        optimizer.step()
        out = torch.max(out, 1)[1]
        running_train += (out == y).float().mean()
    train_acc += (running_train.item() / len(train_dl))
    
    model.eval()
    with torch.no_grad():
        for batch in test_dl:
            x, y = batch
            x, y = x.to(device), y.to(device)
            out = model(x.to(device))
            loss = criterion(out, y.to(device))
            out = torch.max(out, 1)[1]
            running_test += (out == y).float().mean()
        test_acc += (running_test.item() / len(test_dl))
        
train_acc /= epochs
test_acc /= epochs

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:32<00:00, 36.65it/s]


In [88]:
train_acc, test_acc

(0.9596403846154723, 0.9972071428571436)

**Выводы**
* Важно граммотно инициализировать веса (0 - это провал, нормальное распределение (или другое в зависимости от задачи - норм));
* Важно выбрать скорость обучения: при высоком мы будем проскакивать минимум функции, при маленьком модель будет слишком долго сходиться;
* Выбрать количество эпох: модель может либо недообучиться или переобучиться (можно выбрать много, но с динамической скоростью обучения и, возможно, с ранней остановкой);
* Важно определить архитектуру сети: слишком много слоев для простой задачи - плохой выбор, слишком мало для сложной - ничему не научится. Кроме того важно выбрать сами слои и функции активации, сигмоида для скрытых слоев - это провал (затухающие градиенты, например);
* Важно корректно подобрать размер скрытых слоев - зависит от сложности и типа задачи (слишком мало нейронов приведет к тому, что сеть не уловит важных закономерностей, слишком большое - к снижению точности).